#Monthly Leads Assignment
This script is designed to automate the process of distributing renewal leads to sales representatives based on specific assignment rules. It provides a user-friendly interface that allows you to input the names and lead quotas of your sales team members, and then assigns the leads accordingly.

##Purpose
The primary goal of this script is to streamline the lead assignment process and ensure that the sales representatives receive their leads based on predetermined conditions. This helps to maintain a fair distribution of leads and maximizes the efficiency of the sales team.

##How to use
To use this script, simply provide a list of sales representatives along with their desired lead counts, and the script will automatically distribute the leads based on the specified rules.

##Conditional rules
The script follows a set of predefined rules for assigning leads to the sales representatives. These rules are:

- Cindy has priority in receiving leads. She is assigned 500 leads, starting with the Household plans. Any remaining leads up to her quota of 500 are filled with Couple Plan leads.
- Patricia is assigned EDP Dental Plan leads as her primary focus.
- Debbie has a cap on the number of Couple Plan leads she can receive.
- All other sales representatives are assigned the remaining Couple Plan leads, divided equally among them.


Please note that these rules are specific to this implementation and can be customized according to your organization's needs.

## Dependencies

This script requires the following libraries to be installed:

- pandas
- PyQT5
- argparse




In [ ]:
#!pip install PyQT5
#!pip install pandas
#!pip install argparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import argparse
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QLabel, QLineEdit, QPushButton, QFileDialog, QMessageBox
)

def assign_leads(df, dicto, debcop):
    # Sort the leads by customFields_16
    df = df.sort_values('customFields_16')

    # Assign leads to each sales rep
    reps = {}
    leftover = pd.DataFrame(columns=df.columns)
    remaining_couple_leads = df[df['customFields_16'] == 'Couple']
    
    for name, count in dicto.items():
        # Update the remaining_couple_leads DataFrame here
        remaining_couple_leads = df[df['customFields_16'] == 'Couple']

        if name == 'leftover':
            continue
        # Assign Household leads to Cindy
        if name == 'Cindy':
            rep_leads = df[(df['customFields_14'] != 'VSP VISION SAVINGS PASS') & (df['customFields_16'] == 'Household')]
            df = df.drop(rep_leads.index, errors='ignore')
            additional_leads_needed = count - len(rep_leads)
            if additional_leads_needed > 0:
                remaining_couple_leads = df[df['customFields_16'] == 'Couple']
                additional_leads = remaining_couple_leads.sample(additional_leads_needed, random_state=123)
                df = df.drop(additional_leads.index, errors='ignore')
                rep_leads = pd.concat([rep_leads, additional_leads])
                remaining_couple_leads = remaining_couple_leads.drop(remaining_couple_leads.index.intersection(additional_leads.index), errors='ignore')


        # Assign EDP Dental leads to Patricia
        elif name == 'Patricia':
            edp_dental_leads = df[df['customFields_14'] == 'EDP Dental']
            available_leads = min(len(edp_dental_leads), count)
            rep_leads = edp_dental_leads.sample(available_leads, random_state=123)
            df = df.drop(rep_leads.index, errors='ignore')
            additional_leads_needed = count - len(rep_leads)
            if additional_leads_needed > 0:
                remaining_couple_leads = df[df['customFields_16'] == 'Couple']
                additional_leads = df[df['customFields_14'] != 'EDP Dental'].sample(additional_leads_needed, random_state=123)
                df = df.drop(additional_leads.index, errors='ignore')
                rep_leads = pd.concat([rep_leads, additional_leads])
                remaining_couple_leads = remaining_couple_leads.drop(remaining_couple_leads.index.intersection(additional_leads.index), errors='ignore')
        
        # Assign Couple leads to Debbie
        elif name == 'Debbie':
            remaining_couple_leads = df[df['customFields_16'] == 'Couple']
            couple_leads = remaining_couple_leads.sample(debcop, random_state=123)
            single_leads = df[df['customFields_16'] == 'Single'].sample(count - debcop, random_state=123)
            rep_leads = pd.concat([couple_leads, single_leads])
            remaining_couple_leads = remaining_couple_leads.drop(remaining_couple_leads.index.intersection(couple_leads.index), errors='ignore')

        # Assign Single leads to other sales reps
        else:
            remaining_reps = len(dicto) - len(reps)
            remaining_couple_leads = df[df['customFields_16'] == 'Couple']
            couple_count = len(remaining_couple_leads) // remaining_reps
            single_count = count - couple_count

            single_leads = df[df['customFields_16'] == 'Single'].sample(single_count, random_state=123)
            couple_leads = remaining_couple_leads.sample(couple_count, random_state=123)
            remaining_couple_leads = remaining_couple_leads.drop(couple_leads.index, errors='ignore')
            rep_leads = pd.concat([single_leads, couple_leads])
            df = df.drop(single_leads.index, errors='ignore')

        reps[name] = rep_leads
        df = df.drop(rep_leads.index, errors='ignore')

    leftover = df
    # Add leftover DataFrame to reps dictionary
    reps['leftover'] = leftover

    return reps, leftover


def generate_report(reps, leftover):
    report = 'Leads Assignment Report\n\n'
    for name, rep_leads in reps.items():
        report += f'{name}: {len(rep_leads)}\n'
    
    # Check for overlapping member assignments
    overlap = False
    for i, rep1 in enumerate(reps.values()):
        for j, rep2 in enumerate(reps.values()):
            if i >= j:
                continue
            overlap_count = len(set(rep1['MemberId']) & set(rep2['MemberId']))
            if overlap_count > 0:
                report += f'ERROR: {overlap_count} members assigned to both {list(reps.keys())[i]} and {list(reps.keys())[j]}\n'
                overlap = True
    
    if not overlap:
        report += 'No overlapping member assignments\n'
    
    return report

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Leads Assignment')
        self.setGeometry(100, 100, 600, 500)

        # Create input field for the input file path
        self.input_edit = QLineEdit(self)
        self.input_edit.setGeometry(150, 50, 200, 20)  # Increase the width from 200 to 300

        # Create a status label
        self.status_label = QLabel(self)
        self.status_label.setGeometry(150, 80, 400, 20)

        # Create a label to display the number of leads in the file
        self.num_leads_label = QLabel(self)
        self.num_leads_label.setGeometry(50, 400, 400, 20)

        # Create select file button
        self.file_button = QPushButton('Select Input File', self)
        self.file_button.setGeometry(360, 50, 130, 30)
        self.file_button.clicked.connect(self.select_file)

        # Create input for Cindy's leads
        self.cindy_label = QLabel('Cindy:', self)
        self.cindy_label.move(50, 100)
        self.cindy_edit = QLineEdit(self)
        self.cindy_edit.setGeometry(150, 100, 100, 20)

        # Create input for Patricia's leads
        self.patricia_label = QLabel('Patricia:', self)
        self.patricia_label.move(50, 150)
        self.patricia_edit = QLineEdit(self)
        self.patricia_edit.setGeometry(150, 150, 100, 20)

        # Create input for Debbie's leads
        self.debbie_label = QLabel('Debbie:', self)
        self.debbie_label.move(50, 200)
        self.debbie_edit = QLineEdit(self)
        self.debbie_edit.setGeometry(150, 200, 100, 20)

        # Create input for Gina's leads
        self.gina_label = QLabel('Gina:', self)
        self.gina_label.move(50, 250)
        self.gina_edit = QLineEdit(self)
        self.gina_edit.setGeometry(150, 250, 100, 20)

        # Create input for Julie's leads
        self.julie_label = QLabel('Julie:', self)
        self.julie_label.move(50, 300)
        self.julie_edit = QLineEdit(self)
        self.julie_edit.setGeometry(150, 300, 100, 20)

        # Create input for Lisa's leads
        self.lisa_label = QLabel('Lisa:', self)
        self.lisa_label.move(350, 100)
        self.lisa_edit = QLineEdit(self)
        self.lisa_edit.setGeometry(450, 100, 100, 20)

        # Create input for Mimi's leads
        self.mimi_label = QLabel('Mimi:', self)
        self.mimi_label.move(350, 150)
        self.mimi_edit = QLineEdit(self)
        self.mimi_edit.setGeometry(450, 150, 100, 20)

        # Create input for Shaniqua's leads
        self.shaniqua_label = QLabel('Shaniqua:', self)
        self.shaniqua_label.move(350, 200)
        self.shaniqua_edit = QLineEdit(self)
        self.shaniqua_edit.setGeometry(450, 200, 100, 20)

        # Create input for Tammy's leads
        self.tammy_label = QLabel('Tammy:', self)
        self.tammy_label.move(350, 250)
        self.tammy_edit = QLineEdit(self)
        self.tammy_edit.setGeometry(450, 250, 100, 20)

        # Create run button
        self.run_button = QPushButton('Run', self)
        self.run_button.setGeometry(250, 400, 100, 30)
        self.run_button.clicked.connect(self.run_script)

        # Create a label to display the expected number of leads in the 'leftover' campaign
        self.leftover_leads_label = QLabel(self)
        self.leftover_leads_label.setGeometry(50, 350, 400, 20)

    def select_file(self):
        filename, _ = QFileDialog.getOpenFileName(self, "Select input file", "", "Excel Files (*.xlsx)")
        if filename:
            self.input_edit.setText(filename)  # Make sure input_edit is defined in your MainWindow class
            self.status_label.setText('File selected.')

            # Get the number of leads in the file
            df = pd.read_excel(filename)
            num_leads = len(df)
            self.num_leads_label.setText(f'Number of leads in file: {num_leads}')
        else:
            self.status_label.setText('File selection cancelled.')
        
    def run_script(self):
        input_file = self.input_edit.text()

        # Check if input file is provided
        if not input_file:
            QMessageBox.warning(self, 'Warning', 'Please provide an input file.')
            return

        # Get the number of leads in the file
        try:
            df = pd.read_excel(input_file)
            num_leads = len(df)
            self.num_leads_label.setText(f'Number of leads in file: {num_leads}')
        except Exception as e:
            QMessageBox.warning(self, 'Error', f'An error occurred: {str(e)}')
            return

        # Get the lead counts for each sales rep
        try:
            cindy_leads = int(self.cindy_edit.text())
            patricia_leads = int(self.patricia_edit.text())
            debbie_leads = int(self.debbie_edit.text())
            gina_leads = int(self.gina_edit.text())
            julie_leads = int(self.julie_edit.text())
            lisa_leads = int(self.lisa_edit.text())
            mimi_leads = int(self.mimi_edit.text())
            shaniqua_leads = int(self.shaniqua_edit.text())
            tammy_leads = int(self.tammy_edit.text())
            leftover_leads = num_leads - cindy_leads - patricia_leads - debbie_leads - gina_leads - julie_leads - lisa_leads - mimi_leads - shaniqua_leads - tammy_leads
        except ValueError:
            QMessageBox.warning(self, 'Warning', 'Please enter a valid number of leads for each sales rep.')
            return

        # Define dictionary of lead counts for each sales rep
        dicto = {
            'Cindy': cindy_leads,
            'Patricia': patricia_leads,
            'Debbie': debbie_leads,
            'Gina': gina_leads,
            'Julie': julie_leads,
            'Lisa': lisa_leads,
            'Mimi': mimi_leads,
            'Shaniqua': shaniqua_leads,
            'Tammy': tammy_leads,
            'leftover': leftover_leads
        }

        # Update the expected number of leads in the 'leftover' campaign
        leftover_leads = num_leads - cindy_leads - patricia_leads - debbie_leads - gina_leads - julie_leads - lisa_leads - mimi_leads - shaniqua_leads - tammy_leads
        self.leftover_leads_label.setText(f'Expected leads in \'leftover\' campaign: {leftover_leads}')

        # Try to run the script and catch any errors
        try:
            # Load the leads from the input file
            df = pd.read_excel(input_file)

            # Define the number of Couple leads for Debbie
            debcop = 40

            # Assign leads to each sales rep
            reps, leftover = assign_leads(df, dicto, debcop)

            # Save each sales rep's leads to a separate Excel file
            for name, rep_leads in reps.items():
                filename = f'{name}.xlsx'
                rep_leads.to_excel(filename, index=False)

            # Generate a report on the number of leads assigned to each sales rep
            # and any issues with member assignments
            report = generate_report(reps, leftover)
            QMessageBox.information(self, 'Report', report)

        except Exception as e:
            QMessageBox.warning(self, 'Error', f'An error occurred: {str(e)}')



if __name__ == '__main__':
    app = QApplication([])
    window = MainWindow()
    window.show()
    app.exec_()